In [1]:
import os
import functools
from typing import *
from glob import glob
import pickle

import torch
import numpy as np
import pandas as pd

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
#from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE
from sklearn.cluster import KMeans
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import seaborn as sns
from yellowbrick.features import Manifold
from yellowbrick.cluster import KElbowVisualizer, InterclusterDistance, SilhouetteVisualizer
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from altlabs.index_mapping import create_index_mapping
from altlabs.dataset import (
    noop,
    random_roll,
    SoftmaxDataset,
    limit_sequence_size,
    FactorizationDataset,
)
from altlabs.torch.data import FasterBatchSampler, NoAutoCollationDataLoader
from altlabs.utils import Pipeline
from pytorch_lightning import seed_everything
from sklearn.metrics import top_k_accuracy_score
%matplotlib inline

In [2]:
triplet_model_seq_limit_4000_paths = list(sorted(glob("output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/*/checkpoints/*.ckpt")))
lab_index_mapping_seq_limit_4000_paths = list(sorted(glob("output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/*/lab_index_mapping.pkl")))
from altlabs.model.conv1d_triplet_classifier import Conv1dTripletClassifier, ModelConfig

stripping - WARNING - Not using Catalysis: No module named 'catalysis'
output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/0_0/lab_index_mapping.pkl


/home/robot/ds-projects/altlabs_codeocean/code/altlabs/dataset.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self._sequences = np.array([np.array(s) for s in self._sequences])


output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/1_1/lab_index_mapping.pkl
output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/2_2/lab_index_mapping.pkl
output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/3_3/lab_index_mapping.pkl
output/94ccb688-76df-437e-8efc-61ed33bc746f/tensorboard_logs_csv_logs/0_0/lab_index_mapping.pkl


/home/robot/ds-projects/altlabs_codeocean/code/altlabs/dataset.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self._sequences = np.array([np.array(s) for s in self._sequences])


output/94ccb688-76df-437e-8efc-61ed33bc746f/tensorboard_logs_csv_logs/1_1/lab_index_mapping.pkl
output/94ccb688-76df-437e-8efc-61ed33bc746f/tensorboard_logs_csv_logs/2_2/lab_index_mapping.pkl
output/94ccb688-76df-437e-8efc-61ed33bc746f/tensorboard_logs_csv_logs/3_3/lab_index_mapping.pkl
output/c371dd6a-5da5-47e3-82d3-a1e351d5b045/tensorboard_logs_csv_logs/0_0/lab_index_mapping.pkl


/home/robot/ds-projects/altlabs_codeocean/code/altlabs/dataset.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self._sequences = np.array([np.array(s) for s in self._sequences])


output/c371dd6a-5da5-47e3-82d3-a1e351d5b045/tensorboard_logs_csv_logs/1_1/lab_index_mapping.pkl
output/c371dd6a-5da5-47e3-82d3-a1e351d5b045/tensorboard_logs_csv_logs/2_2/lab_index_mapping.pkl
output/c371dd6a-5da5-47e3-82d3-a1e351d5b045/tensorboard_logs_csv_logs/3_3/lab_index_mapping.pkl
output/3d70caa8-06d2-4247-a663-53c3be194a0d/tensorboard_logs_csv_logs/0_0/lab_index_mapping.pkl


/home/robot/ds-projects/altlabs_codeocean/code/altlabs/dataset.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self._sequences = np.array([np.array(s) for s in self._sequences])


output/3d70caa8-06d2-4247-a663-53c3be194a0d/tensorboard_logs_csv_logs/1_1/lab_index_mapping.pkl
output/3d70caa8-06d2-4247-a663-53c3be194a0d/tensorboard_logs_csv_logs/2_2/lab_index_mapping.pkl


In [3]:
lab_index_mapping_seq_limit_4000_paths

['output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/0_0/lab_index_mapping.pkl',
 'output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/1_1/lab_index_mapping.pkl',
 'output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/2_2/lab_index_mapping.pkl',
 'output/b4a3cfe8-8678-4253-a9fa-62d7a5e300a8/tensorboard_logs_csv_logs/3_3/lab_index_mapping.pkl']

In [4]:
DATA_PATH = "../data/"

In [5]:
device = torch.device("cuda:0")

train_values_df = pd.read_csv(f"{DATA_PATH}train_values_grouped.csv")
format_df = pd.read_csv(f"{DATA_PATH}format.csv")
test_values_df = pd.read_csv(f"{DATA_PATH}test_values.csv")
test_set = pd.read_csv(f"{DATA_PATH}test_labels.csv")
pub_id = pd.read_csv(f"{DATA_PATH}pubsubidx.csv")

In [6]:
pub_index = pub_id[pub_id.public==True].index
private_index = pub_id[pub_id.public==False].index

In [7]:
seed_everything(350)
def predict_dataset(model: Conv1dTripletClassifier, dataset: FactorizationDataset, tta_steps: int) -> np.ndarray:
    batch_sampler = FasterBatchSampler(
        dataset, 32, shuffle=False,
    )

    model.to(device)

    predictions: List[List[float]] = []
    with torch.no_grad():
        for indices in batch_sampler:
            if tta_steps > 0:
                tta_predictions = []
                for i in range(tta_steps):
                    batch = dataset[indices]
                    if isinstance(batch[0], tuple):
                        (sequences, extra_inputs, _) = batch[
                            0
                        ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                    else:
                        (sequences, extra_inputs) = batch
                    outputs = model.predict_lab_scores(
                        sequences.to(device), extra_inputs.to(device)
                    ).tolist()
                    tta_predictions.append(np.array(outputs))
                predictions.extend(
                    np.mean(np.array(tta_predictions), axis=0).tolist()
                )
            else:
                batch = dataset[indices]
                if isinstance(batch[0], tuple):
                    (sequences, extra_inputs, _) = batch[
                        0
                    ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                else:
                    (sequences, extra_inputs) = batch
                outputs = model.predict_lab_scores(
                    sequences.to(device), extra_inputs.to(device)
                ).tolist()
                predictions.extend(outputs)

    return np.array(predictions)

sequence_index_mapping = create_index_mapping(
"ATGC", include_unkown=True, include_none=False,
)
sequence_index_mapping["N"] = 0
input_columns = train_values_df.drop(columns=["sequence_id", "groups", "output"]).columns

fold_output = []
for triplet_model_path, lab_index_mapping_path in zip(triplet_model_seq_limit_4000_paths, lab_index_mapping_seq_limit_4000_paths):
    print(lab_index_mapping_path)
    model = Conv1dTripletClassifier.load_from_checkpoint(triplet_model_path)
    with open(lab_index_mapping_path, "rb") as f:
        lab_index_mapping = pickle.load(f)
    dataset = FactorizationDataset(
        test_values_df,
        sequence_index_mapping,
        lab_index_mapping,
        input_columns,
        lab_column="output",
        negative_proportion=0.0,
        transform_sequence_fn=random_roll,
        test=True,
        bpe=True,
    )

    outputs = predict_dataset(model, dataset, 10)
    fold_output.append(outputs)
final_outputs = np.mean(fold_output, axis=0)

for lab in format_df.columns[1:]:
    lab_index = lab_index_mapping[lab]
    format_df[lab] = final_outputs[:, lab_index]

format_df = format_df.round(6)


    

In [8]:
pub_best_sub = format_df[format_df.index.isin(pub_index)]
private_best_sub = format_df[format_df.index.isin(private_index)]
private_test = test_set[test_set.index.isin(private_index)]
public_test = test_set[test_set.index.isin(pub_index)]
private_labels = private_test.drop(columns="sequence_id").values.argmax(axis=1)
public_labels = public_test.drop(columns="sequence_id").values.argmax(axis=1)

top_10_score_private_4000 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_private_4000 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_public_4000 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_public_4000 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_total_4000 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_total_4000 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

display(f"Triplet model top-10 private score: {top_10_score_private_4000}")

display(f"Triplet model top-1 private score: {top_1_score_private_4000}")


display(f"Triplet model top-10 public score: {top_10_score_public_4000}")

display(f"Triplet model top-1 public score: {top_1_score_public_4000}")

display(f"Triplet model top-10 total score: {top_10_score_total_4000}")

display(f"Triplet model top-1 total score: {top_1_score_total_4000}")

'Triplet model top-10 private score: 0.8670601709100519'

'Triplet model top-1 private score: 0.6856664611047485'

'Triplet model top-10 public score: 0.8821165438713998'

'Triplet model top-1 public score: 0.7402545210984595'

'Triplet model top-10 total score: 0.8730335884353742'

'Triplet model top-1 total score: 0.7073235544217688'

In [9]:
triplet_model_seq_limit_3000_paths = list(sorted(glob("output/94ccb688-76df-437e-8efc-61ed33bc746f/tensorboard_logs_csv_logs/*/checkpoints/*.ckpt")))
lab_index_mapping_seq_limit_3000_paths = list(sorted(glob("output/94ccb688-76df-437e-8efc-61ed33bc746f/tensorboard_logs_csv_logs/*/lab_index_mapping.pkl")))


In [10]:
seed_everything(350)
def predict_dataset(model: Conv1dTripletClassifier, dataset: FactorizationDataset, tta_steps: int) -> np.ndarray:
    batch_sampler = FasterBatchSampler(
        dataset, 32, shuffle=False,
    )

    model.to(device)

    predictions: List[List[float]] = []
    with torch.no_grad():
        for indices in batch_sampler:
            if tta_steps > 0:
                tta_predictions = []
                for i in range(tta_steps):
                    batch = dataset[indices]
                    if isinstance(batch[0], tuple):
                        (sequences, extra_inputs, _) = batch[
                            0
                        ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                    else:
                        (sequences, extra_inputs) = batch
                    outputs = model.predict_lab_scores(
                        sequences.to(device), extra_inputs.to(device)
                    ).tolist()
                    tta_predictions.append(np.array(outputs))
                predictions.extend(
                    np.mean(np.array(tta_predictions), axis=0).tolist()
                )
            else:
                batch = dataset[indices]
                if isinstance(batch[0], tuple):
                    (sequences, extra_inputs, _) = batch[
                        0
                    ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                else:
                    (sequences, extra_inputs) = batch
                outputs = model.predict_lab_scores(
                    sequences.to(device), extra_inputs.to(device)
                ).tolist()
                predictions.extend(outputs)

    return np.array(predictions)

sequence_index_mapping = create_index_mapping(
"ATGC", include_unkown=True, include_none=False,
)
sequence_index_mapping["N"] = 0
input_columns = train_values_df.drop(columns=["sequence_id", "groups", "output"]).columns

fold_output = []
for triplet_model_path, lab_index_mapping_path in zip(triplet_model_seq_limit_3000_paths, lab_index_mapping_seq_limit_3000_paths):
    print(lab_index_mapping_path)
    model = Conv1dTripletClassifier.load_from_checkpoint(triplet_model_path)
    with open(lab_index_mapping_path, "rb") as f:
        lab_index_mapping = pickle.load(f)
    dataset = FactorizationDataset(
        test_values_df,
        sequence_index_mapping,
        lab_index_mapping,
        input_columns,
        lab_column="output",
        negative_proportion=0.0,
        transform_sequence_fn=random_roll,
        test=True,
        bpe=True,
    )

    outputs = predict_dataset(model, dataset, 10)
    fold_output.append(outputs)
final_outputs = np.mean(fold_output, axis=0)

for lab in format_df.columns[1:]:
    lab_index = lab_index_mapping[lab]
    format_df[lab] = final_outputs[:, lab_index]

format_df = format_df.round(6)


    

In [11]:
pub_best_sub = format_df[format_df.index.isin(pub_index)]
private_best_sub = format_df[format_df.index.isin(private_index)]
private_test = test_set[test_set.index.isin(private_index)]
public_test = test_set[test_set.index.isin(pub_index)]
private_labels = private_test.drop(columns="sequence_id").values.argmax(axis=1)
public_labels = public_test.drop(columns="sequence_id").values.argmax(axis=1)

top_10_score_private_3000 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_private_3000 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_public_3000 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_public_3000 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_total_3000 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_total_3000 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

display(f"Triplet model top-10 private score: {top_10_score_private_3000}")

display(f"Triplet model top-1 private score: {top_1_score_private_3000}")


display(f"Triplet model top-10 public score: {top_10_score_public_3000}")

display(f"Triplet model top-1 public score: {top_1_score_public_3000}")

display(f"Triplet model top-10 total score: {top_10_score_total_3000}")

display(f"Triplet model top-1 total score: {top_1_score_total_3000}")

'Triplet model top-10 private score: 0.8682935424191701'

'Triplet model top-1 private score: 0.6920095145802132'

'Triplet model top-10 public score: 0.8755525787006028'

'Triplet model top-1 public score: 0.7438713998660416'

'Triplet model top-10 total score: 0.8711734693877551'

'Triplet model top-1 total score: 0.7125850340136054'

In [12]:
triplet_model_seq_limit_2000_paths = list(sorted(glob("output/c371dd6a-5da5-47e3-82d3-a1e351d5b045/tensorboard_logs_csv_logs/*/checkpoints/*.ckpt")))
lab_index_mapping_seq_limit_2000_paths = list(sorted(glob("output/c371dd6a-5da5-47e3-82d3-a1e351d5b045/tensorboard_logs_csv_logs/*/lab_index_mapping.pkl")))


In [13]:
seed_everything(350)
def predict_dataset(model: Conv1dTripletClassifier, dataset: FactorizationDataset, tta_steps: int) -> np.ndarray:
    batch_sampler = FasterBatchSampler(
        dataset, 32, shuffle=False,
    )

    model.to(device)

    predictions: List[List[float]] = []
    with torch.no_grad():
        for indices in batch_sampler:
            if tta_steps > 0:
                tta_predictions = []
                for i in range(tta_steps):
                    batch = dataset[indices]
                    if isinstance(batch[0], tuple):
                        (sequences, extra_inputs, _) = batch[
                            0
                        ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                    else:
                        (sequences, extra_inputs) = batch
                    outputs = model.predict_lab_scores(
                        sequences.to(device), extra_inputs.to(device)
                    ).tolist()
                    tta_predictions.append(np.array(outputs))
                predictions.extend(
                    np.mean(np.array(tta_predictions), axis=0).tolist()
                )
            else:
                batch = dataset[indices]
                if isinstance(batch[0], tuple):
                    (sequences, extra_inputs, _) = batch[
                        0
                    ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                else:
                    (sequences, extra_inputs) = batch
                outputs = model.predict_lab_scores(
                    sequences.to(device), extra_inputs.to(device)
                ).tolist()
                predictions.extend(outputs)

    return np.array(predictions)

sequence_index_mapping = create_index_mapping(
"ATGC", include_unkown=True, include_none=False,
)
sequence_index_mapping["N"] = 0
input_columns = train_values_df.drop(columns=["sequence_id", "groups", "output"]).columns

fold_output = []
for triplet_model_path, lab_index_mapping_path in zip(triplet_model_seq_limit_2000_paths, lab_index_mapping_seq_limit_2000_paths):
    print(lab_index_mapping_path)
    model = Conv1dTripletClassifier.load_from_checkpoint(triplet_model_path)
    with open(lab_index_mapping_path, "rb") as f:
        lab_index_mapping = pickle.load(f)
    dataset = FactorizationDataset(
        test_values_df,
        sequence_index_mapping,
        lab_index_mapping,
        input_columns,
        lab_column="output",
        negative_proportion=0.0,
        transform_sequence_fn=random_roll,
        test=True,
        bpe=True,
    )

    outputs = predict_dataset(model, dataset, 10)
    fold_output.append(outputs)
final_outputs = np.mean(fold_output, axis=0)

for lab in format_df.columns[1:]:
    lab_index = lab_index_mapping[lab]
    format_df[lab] = final_outputs[:, lab_index]

format_df = format_df.round(6)


    

In [14]:
pub_best_sub = format_df[format_df.index.isin(pub_index)]
private_best_sub = format_df[format_df.index.isin(private_index)]
private_test = test_set[test_set.index.isin(private_index)]
public_test = test_set[test_set.index.isin(pub_index)]
private_labels = private_test.drop(columns="sequence_id").values.argmax(axis=1)
public_labels = public_test.drop(columns="sequence_id").values.argmax(axis=1)

top_10_score_private_2000 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_private_2000 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_public_2000 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_public_2000 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_total_2000 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_total_2000 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

display(f"Triplet model top-10 private score: {top_10_score_private_2000}")

display(f"Triplet model top-1 private score: {top_1_score_private_2000}")


display(f"Triplet model top-10 public score: {top_10_score_public_2000}")

display(f"Triplet model top-1 public score: {top_1_score_public_2000}")

display(f"Triplet model top-10 total score: {top_10_score_total_2000}")

display(f"Triplet model top-1 total score: {top_1_score_total_2000}")

'Triplet model top-10 private score: 0.867676856664611'

'Triplet model top-1 private score: 0.690071359351599'

'Triplet model top-10 public score: 0.883456128600134'

'Triplet model top-1 public score: 0.7438713998660416'

'Triplet model top-10 total score: 0.873937074829932'

'Triplet model top-1 total score: 0.7114158163265306'

In [15]:
triplet_model_seq_limit_500_paths = list(sorted(glob("output/3d70caa8-06d2-4247-a663-53c3be194a0d/tensorboard_logs_csv_logs/*/checkpoints/*.ckpt")))
lab_index_mapping_seq_limit_500_paths = list(sorted(glob("output/3d70caa8-06d2-4247-a663-53c3be194a0d/tensorboard_logs_csv_logs/*/lab_index_mapping.pkl")))


In [ ]:
seed_everything(350)
def predict_dataset(model: Conv1dTripletClassifier, dataset: FactorizationDataset, tta_steps: int) -> np.ndarray:
    batch_sampler = FasterBatchSampler(
        dataset, 32, shuffle=False,
    )

    model.to(device)

    predictions: List[List[float]] = []
    with torch.no_grad():
        for indices in batch_sampler:
            if tta_steps > 0:
                tta_predictions = []
                for i in range(tta_steps):
                    batch = dataset[indices]
                    if isinstance(batch[0], tuple):
                        (sequences, extra_inputs, _) = batch[
                            0
                        ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                    else:
                        (sequences, extra_inputs) = batch
                    outputs = model.predict_lab_scores(
                        sequences.to(device), extra_inputs.to(device)
                    ).tolist()
                    tta_predictions.append(np.array(outputs))
                predictions.extend(
                    np.mean(np.array(tta_predictions), axis=0).tolist()
                )
            else:
                batch = dataset[indices]
                if isinstance(batch[0], tuple):
                    (sequences, extra_inputs, _) = batch[
                        0
                    ]  # type: (torch.Tensor, torch.Tensor, torch.Tensor)
                else:
                    (sequences, extra_inputs) = batch
                outputs = model.predict_lab_scores(
                    sequences.to(device), extra_inputs.to(device)
                ).tolist()
                predictions.extend(outputs)

    return np.array(predictions)

sequence_index_mapping = create_index_mapping(
"ATGC", include_unkown=True, include_none=False,
)
sequence_index_mapping["N"] = 0
input_columns = train_values_df.drop(columns=["sequence_id", "groups", "output"]).columns

fold_output = []
for triplet_model_path, lab_index_mapping_path in zip(triplet_model_seq_limit_500_paths, lab_index_mapping_seq_limit_500_paths):
    print(lab_index_mapping_path)
    model = Conv1dTripletClassifier.load_from_checkpoint(triplet_model_path)
    with open(lab_index_mapping_path, "rb") as f:
        lab_index_mapping = pickle.load(f)
    dataset = FactorizationDataset(
        test_values_df,
        sequence_index_mapping,
        lab_index_mapping,
        input_columns,
        lab_column="output",
        negative_proportion=0.0,
        transform_sequence_fn=random_roll,
        test=True,
        bpe=True,
    )

    outputs = predict_dataset(model, dataset, 10)
    fold_output.append(outputs)
final_outputs = np.mean(fold_output, axis=0)

for lab in format_df.columns[1:]:
    lab_index = lab_index_mapping[lab]
    format_df[lab] = final_outputs[:, lab_index]

format_df = format_df.round(6)


    

In [ ]:
pub_best_sub = format_df[format_df.index.isin(pub_index)]
private_best_sub = format_df[format_df.index.isin(private_index)]
private_test = test_set[test_set.index.isin(private_index)]
public_test = test_set[test_set.index.isin(pub_index)]
private_labels = private_test.drop(columns="sequence_id").values.argmax(axis=1)
public_labels = public_test.drop(columns="sequence_id").values.argmax(axis=1)

top_10_score_private_500 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_private_500 = top_k_accuracy_score(private_labels, private_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_public_500 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_public_500 = top_k_accuracy_score(public_labels, pub_best_sub.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

top_10_score_total_500 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=10, labels=range(0,1314))

top_1_score_total_500 = top_k_accuracy_score(test_set.drop(columns="sequence_id").values.argmax(axis=1), format_df.drop(columns="sequence_id").values, k=1, labels=range(0,1314))

display(f"Triplet model top-10 private score: {top_10_score_private_500}")

display(f"Triplet model top-1 private score: {top_1_score_private_500}")


display(f"Triplet model top-10 public score: {top_10_score_public_500}")

display(f"Triplet model top-1 public score: {top_1_score_public_500}")

display(f"Triplet model top-10 total score: {top_10_score_total_500}")

display(f"Triplet model top-1 total score: {top_1_score_total_500}")

In [19]:
display(f"Triplet model top-1 total score: {top_1_score_total_500}")

'Triplet model top-1 total score: 0.7242240646258503'

In [22]:
import os
RESULTS_PATH = "../results/"

if os.path.exists(f"{RESULTS_PATH}results.csv"):
    result_df = pd.read_csv(f"{RESULTS_PATH}results.csv")
else:
    result_df = pd.DataFrame([], columns=["Model", "Top 10 Score", "Top 1 Score", "Top 10 New Data Score", "Top 1 New Data Score", "Top 10 Total Score", "Top 1 Total Score"])
    

result_df = result_df.append({"Model": "BPE - Seq Limit 4000", 
                              "Top 10 Score": top_10_score_private_4000, "Top 1 Score": top_1_score_private_4000, 
                              "Top 10 New Data Score": top_10_score_public_4000, "Top 1 New Data Score": top_1_score_public_4000,
                             "Top 10 Total Score": top_10_score_total_4000, "Top 1 Total Score": top_1_score_total_4000}, ignore_index=True)

result_df = result_df.append({"Model": "BPE - Seq Limit 3000", 
                              "Top 10 Score": top_10_score_private_3000, "Top 1 Score": top_1_score_private_3000, 
                              "Top 10 New Data Score": top_10_score_public_3000, "Top 1 New Data Score": top_1_score_public_3000,
                             "Top 10 Total Score": top_10_score_total_3000, "Top 1 Total Score": top_1_score_total_3000}, ignore_index=True)


result_df = result_df.append({"Model": "BPE - Seq Limit 2000", 
                              "Top 10 Score": top_10_score_private_2000, "Top 1 Score": top_1_score_private_2000, 
                              "Top 10 New Data Score": top_10_score_public_2000, "Top 1 New Data Score": top_1_score_public_2000,
                             "Top 10 Total Score": top_10_score_total_2000, "Top 1 Total Score": top_1_score_total_2000}, ignore_index=True)


result_df = result_df.append({"Model": "BPE - Seq Limit 500", 
                              "Top 10 Score": top_10_score_private_500, "Top 1 Score": top_1_score_private_500, 
                              "Top 10 New Data Score": top_10_score_public_500, "Top 1 New Data Score": top_1_score_public_500,
                             "Top 10 Total Score": top_10_score_total_500, "Top 1 Total Score": top_1_score_total_500}, ignore_index=True)


result_df.to_csv(f"{RESULTS_PATH}results.csv", index=False)